In [103]:
import gurobipy as gp
from gurobipy import GRB

import numpy as np
import random

## Data

In [104]:
n = 5
subsets_config = [2, 2, 3, 1, 5] 

In [105]:
base_numbers = list(range(n))
def generating_data(n, subsets_config):
    random.seed(42)

    
    while True:
        subsets = []

        for size in subsets_config:
            if size > n:
                print("Size cannot be larger than n")
            subset = random.sample(base_numbers, size)
            subsets.append(subset)

        union = set()
        for subset in subsets:
            union.update(subset)
        
        if union == set(base_numbers):
            break


    # A = np.random.rand(n, n)
    A = np.random.uniform(-1, 1, size=(n, n)) 
    A = np.triu(A)
    
    return subsets, A

subsets, A = generating_data(5, subsets_config)

In [106]:
subsets

[[0, 4], [2, 1], [1, 4, 2], [0], [4, 0, 2, 1, 3]]

In [107]:
A

array([[-5.42729080e-01, -3.17081445e-01, -9.97251352e-01,
        -9.30733642e-01,  4.75416297e-01],
       [ 0.00000000e+00, -8.44398797e-03,  6.13022131e-01,
         6.46498392e-01, -8.74034119e-02],
       [ 0.00000000e+00,  0.00000000e+00,  3.17289022e-01,
         9.87310042e-01, -8.00671047e-01],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         4.82776277e-04,  4.21144359e-01],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  6.29188284e-01]])

In [108]:
e = {(k,i): 1 if k in subsets[i] else 0 
     for k in base_numbers 
     for i in range(len(subsets))}
e

{(0, 0): 1,
 (0, 1): 0,
 (0, 2): 0,
 (0, 3): 1,
 (0, 4): 1,
 (1, 0): 0,
 (1, 1): 1,
 (1, 2): 1,
 (1, 3): 0,
 (1, 4): 1,
 (2, 0): 0,
 (2, 1): 1,
 (2, 2): 1,
 (2, 3): 0,
 (2, 4): 1,
 (3, 0): 0,
 (3, 1): 0,
 (3, 2): 0,
 (3, 3): 0,
 (3, 4): 1,
 (4, 0): 1,
 (4, 1): 0,
 (4, 2): 1,
 (4, 3): 0,
 (4, 4): 1}

## Model

In [109]:
model = gp.Model("MAX-SC-QBF")

x = model.addVars(n, vtype=GRB.BINARY, name="x_i")
y = model.addVars(n, n, vtype=GRB.BINARY, name="y")

obj = gp.QuadExpr()
for i in range(n):
    for j in range(n):
        obj += A[i][j] * y[i,j]

model.setObjective(obj, GRB.MAXIMIZE)

model.addConstrs((y[i,j] <= x[i] for i in range(n) for j in range(n)), "constrain_1")
model.addConstrs((y[i,j] <= x[j] for i in range(n) for j in range(n)), "constrain_2")
model.addConstrs((y[i,j] >= x[i] + x[j] - 1 for i in range(n) for j in range(n)), "constrain_3")

for k in base_numbers:
    model.addConstr(
        gp.quicksum(e[k,i] * x[i] for i in range(len(subsets))) >= 1,
        name=f"cover_{k}"
    )
    
model.setParam('TimeLimit', 60*10)
model.optimize()

Set parameter TimeLimit to value 600
Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (mac64[arm] - Darwin 24.6.0 24G84)

CPU model: Apple M3 Pro
Thread count: 12 physical cores, 12 logical processors, using up to 12 threads

Non-default parameters:
TimeLimit  600

Optimize a model with 80 rows, 30 columns and 183 nonzeros
Model fingerprint: 0xa38c79a1
Variable types: 0 continuous, 30 integer (30 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+00]
  Objective range  [5e-04, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 0.4060373
Found heuristic solution: objective 0.6291883
Presolve removed 80 rows and 30 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.00 seconds (0.00 work units)
Thread count was 1 (of 12 available processors)

Solution count 3: 2.71842 0.629188 0.406037 

Optimal solution found (tolerance 1.00e-04)
Best objective 2.718416559

## Results

In [110]:
if model.status == GRB.OPTIMAL:
    print("\nFinal Solution:")
    print(f"Optimal Objective Value: {model.ObjVal}")

    print("\nSelected Sets:")
    for i in range(len(subsets)):
        if x[i].X > 0.5:
            print(f"Set {i} (Cover elements {subsets[i]})")
    
    print("\nCoverage verification:")
    for v in base_numbers:
        covered = any(x[i].X > 0.5 for i in range(len(subsets)) if v in subsets[i])
        print(f"Value {v}: {'Covered' if covered else 'NOT COVERED'}")
else:
    print("No solution found")


Final Solution:
Optimal Objective Value: 2.7184165594395577

Selected Sets:
Set 1 (Cover elements [2, 1])
Set 2 (Cover elements [1, 4, 2])
Set 3 (Cover elements [0])
Set 4 (Cover elements [4, 0, 2, 1, 3])

Coverage verification:
Value 0: Covered
Value 1: Covered
Value 2: Covered
Value 3: Covered
Value 4: Covered
